#### 원핫벡터
예시) [['강아지'], ['고양이'], ['냉장고']] 분류
- 강아지, 고양이, 냉장고를 [[1],[2],[3]] 로 레이블 후 분류
    - 각 클래스들간의 관계 생성(유사성)
    - 균등하게 분류x
- 원-핫 인코딩으로 [[1,0,0],[0,1,0],[0,0,1]] 로 레이블 후 분류
    - 각 클래스들을 균등하게 분류
    - 각 클래스간의 유사성x

## 다중 클래스 분류

#### 소프트맥스 회귀

$H(X) = softmax(WX + B)$

$\mathrm{softmax}(\mathbf{z}) = \left[ \frac{e^{z_1}}{\sum_{j=1}^{3} e^{z_j}}, \frac{e^{z_2}}{\sum_{j=1}^{3} e^{z_j}}, \frac{e^{z_3}}{\sum_{j=1}^{3} e^{z_j}} \right] = [p_1, p_2, p_3] = \hat{y}$


- 소프트맥스를 지난 각 원소들의 합은 1

#### 손실 함수
- 크로스 엔트로피 함수

$cost(W) = -\displaystyle\sum_{j=1}^{k}{y_j\log(p_j)}$

$cost(W) = -\displaystyle \frac{1}{n}\displaystyle\sum_{i=1}^{n}\sum_{j=1}^{k}{y_j}^{(i)} \times (-\log({p_j}^{(i)}))$

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [ ]:
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim=0) #dim은 몇번째 차원에 대하여 소프트맥스를 적용할건지 정의
hypothesis

tensor([0.0900, 0.2447, 0.6652])

In [14]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
hypothesis

tensor([[0.2441, 0.1429, 0.2298, 0.2344, 0.1487],
        [0.1665, 0.2504, 0.2309, 0.1707, 0.1815],
        [0.2733, 0.1576, 0.2292, 0.2147, 0.1252]], grad_fn=<SoftmaxBackward0>)

In [15]:
hypothesis.sum()

tensor(3.0000, grad_fn=<SumBackward0>)

In [29]:
y = torch.randint(5, (3,)).long()
y

tensor([3, 4, 3])

In [35]:
y_one_hot = torch.zeros_like(hypothesis)
#scatter_(dim,index,value)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.]])

In [36]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
cost

tensor(1.5653, grad_fn=<MeanBackward0>)

In [39]:
print(F.log_softmax(z,dim=1))
print(torch.log(F.softmax(z, dim=1)))

tensor([[-1.4101, -1.9457, -1.4703, -1.4507, -1.9056],
        [-1.7925, -1.3848, -1.4657, -1.7679, -1.7066],
        [-1.2971, -1.8475, -1.4731, -1.5385, -2.0782]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[-1.4101, -1.9457, -1.4703, -1.4507, -1.9056],
        [-1.7925, -1.3848, -1.4657, -1.7679, -1.7066],
        [-1.2971, -1.8475, -1.4731, -1.5385, -2.0782]], grad_fn=<LogBackward0>)


In [41]:
print((y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean())
print((y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean())
print(F.nll_loss(F.log_softmax(z, dim=1), y))
print(F.cross_entropy(z, y))

tensor(1.5653, grad_fn=<MeanBackward0>)
tensor(1.5653, grad_fn=<MeanBackward0>)
tensor(1.5653, grad_fn=<NllLossBackward0>)
tensor(1.5653, grad_fn=<NllLossBackward0>)


In [51]:
import torch.nn as nn
import torch.optim as optim

In [43]:
critierion = nn.CrossEntropyLoss()
loss = critierion(z, y)
print(loss)

tensor(1.5653, grad_fn=<NllLossBackward0>)


#### 다중 클래스 분류 데이터)

SepalLengthCm(x_1)|SepalWidthCm(x_2)|PetalLengthCm(x_3)|PetalWidthCm(x_4)|Species(y)
-|-|-|-|-
5.1|3.5|1.4|0.2|setosa
4.9|3.0|1.4|0.2|setosa
5.8|2.6|4.0|1.2|versicolor
6.7|3.0|5.2|2.3|virginica
5.6|2.8|4.9|2.0|virginica

In [ ]:
setosa = 0
versicolor = 1
virginica = 2

x_train = [[5.1, 3.5, 1.4, 0.2],
           [4.9, 3.0, 1.4, 0.2],
           [5.8, 2.6, 4.0, 1.2],
           [6.7, 3.0, 5.2, 2.3],
           [5.6, 2.8, 4.9, 2.0]]
y_train = [setosa, setosa, versicolor, virginica, virginica]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

x_train.shape ,y_train.shape

(torch.Size([5, 4]), torch.Size([5]))

In [50]:
y_one_hot = torch.zeros(5, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
y_one_hot.shape

torch.Size([5, 3])

In [54]:
model = nn.Linear(4,3)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [55]:
epochs = 1000

for epoch in range(epochs):
    prediction = model(x_train)
    cost = loss_function(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()


    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.077278
Epoch  100/1000 Cost: 0.234721
Epoch  200/1000 Cost: 0.155795
Epoch  300/1000 Cost: 0.115142
Epoch  400/1000 Cost: 0.090596
Epoch  500/1000 Cost: 0.074347
Epoch  600/1000 Cost: 0.062876
Epoch  700/1000 Cost: 0.054383
Epoch  800/1000 Cost: 0.047860
Epoch  900/1000 Cost: 0.042703


In [56]:
with torch.no_grad():
    new_x = [5.0,3.2,1.4,0.2]
    new_x = torch.FloatTensor(new_x)
    new_prediction = model(new_x)
    print(new_prediction)

tensor([ 6.7816,  2.1472, -6.9098])
